In [1]:
# Streamlit 설치
!pip install streamlit -q
# Ultralytics YOLOv8 라이브러리 설치
!pip install ultralytics -q
# PIL(Pillow) 설치 (이미지 처리 기본 라이브러리)
!pip install Pillow -q
!pip install streamlit pyngrok -q

from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.6 MB/s eta 0:00:00


ValueError: mount failed

## 섹션 2 팀 application 전체 코드

In [2]:
%%writefile pill_detection_ollama_appl.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image
import cv2
import numpy as np
import tempfile
import os
import requests
import json
import io
import yaml

# 페이지 설정
st.set_page_config(
    page_title="약 검출 및 상호작용 분석 시스템", page_icon="💊", layout="wide"
)

# 제목
st.title("약 검출 및 상호작용 분석 시스템")
st.markdown("YOLO 모델로 약을 검출하고, Ollama를 통해 약물 상호작용을 분석합니다.")

# 사이드바
st.sidebar.header("설정")

# 신뢰도 임계값 설정
confidence_threshold = st.sidebar.slider(
    "검출 신뢰도 임계값", min_value=0.0, max_value=1.0, value=0.25, step=0.05
)

# 클래스 이름 설정
st.sidebar.subheader("약물 클래스 이름 설정")

class_name_option = st.sidebar.radio(
    "클래스명 설정 방식",
    ["모델 기본값", "YAML/JSON 파일 업로드", "직접 입력"],
    help="약물명을 설정하는 방법을 선택하세요",
)

class_names_dict = {}

if class_name_option == "YAML/JSON 파일 업로드":
    uploaded_class_file = st.sidebar.file_uploader(
        "클래스 파일 업로드",
        type=["yaml", "yml", "json"],
        help="YOLO YAML 파일 또는 클래스 매핑 JSON 파일을 업로드하세요",
    )

    if uploaded_class_file is not None:
        try:
            file_extension = uploaded_class_file.name.split(".")[-1].lower()

            if file_extension in ["yaml", "yml"]:
                # YAML 파일 파싱
                yaml_content = yaml.safe_load(uploaded_class_file)
                if "names" in yaml_content:
                    class_names_list = yaml_content["names"]
                    class_names_dict = {
                        i: str(name) for i, name in enumerate(class_names_list)
                    }
                    st.sidebar.success(
                        f"YAML에서 {len(class_names_dict)}개 클래스 로드 완료"
                    )
                else:
                    st.sidebar.error("YAML 파일에 'names' 필드가 없습니다")

            elif file_extension == "json":
                # JSON 파일 파싱
                json_content = json.load(uploaded_class_file)

                # class_mapping.json 형식 처리 (약물코드: {index: N})
                if all(
                    isinstance(v, dict) and "index" in v for v in json_content.values()
                ):
                    for drug_code, info in json_content.items():
                        class_names_dict[info["index"]] = drug_code
                    st.sidebar.success(
                        f"JSON에서 {len(class_names_dict)}개 클래스 로드 완료"
                    )
                # 일반 매핑 형식 처리 {index: name}
                else:
                    class_names_dict = {int(k): str(v) for k, v in json_content.items()}
                    st.sidebar.success(
                        f"JSON에서 {len(class_names_dict)}개 클래스 로드 완료"
                    )

            # 매핑 확인
            if class_names_dict:
                with st.sidebar.expander("매핑 확인 (처음 10개)"):
                    for idx in sorted(list(class_names_dict.keys())[:10]):
                        st.text(f"Class {idx} → {class_names_dict[idx]}")
                    if len(class_names_dict) > 10:
                        st.text(f"... 외 {len(class_names_dict) - 10}개")

        except Exception as e:
            st.sidebar.error(f"파일 로드 실패: {e}")

elif class_name_option == "직접 입력":
    st.sidebar.info("쉼표로 구분하여 약물명을 입력하세요")

    class_names_input = st.sidebar.text_area(
        "약물명 입력 (순서대로)",
        value="약물1, 약물2, 약물3",
        help="클래스 0부터 순서대로 약물명을 입력하세요. 쉼표로 구분합니다.",
        height=100,
    )

    # 입력된 약물명 파싱
    if class_names_input:
        class_names_list = [name.strip() for name in class_names_input.split(",")]
        class_names_dict = {i: name for i, name in enumerate(class_names_list)}

        with st.sidebar.expander("매핑 확인"):
            for idx, name in class_names_dict.items():
                st.text(f"Class {idx} → {name}")

# 약물명 한글 변환 옵션 (약물 식별번호 -> 실제 약물명)
if class_names_dict:
    st.sidebar.markdown("---")
    convert_to_korean = st.sidebar.checkbox(
        "약물 식별번호를 실제 약물명으로 변환",
        value=False,
        help="e약은요 API를 사용하여 약물코드를 실제 약물명으로 변환합니다",
    )

    if convert_to_korean:
        st.sidebar.info("한국의약품안전나라 API 키가 필요합니다")

        api_key_input = st.sidebar.text_input(
            "API 키 입력",
            value="d4f215cfeaa74bb71b94270e1bc07cbe217bde62d91a4b0c094d2f059b2dec9c",
            type="password",
            help="https://www.data.go.kr 에서 발급받은 API 키를 입력하세요",
        )

        if api_key_input:
            # 세션 상태에 API 키 저장
            st.session_state["drug_api_key"] = api_key_input

            # 변환 버튼
            if st.sidebar.button("약물명 변환 시작"):
                # 현재 클래스 이름에서 약물 식별번호 추출
                drug_codes = list(set(class_names_dict.values()))

                # API로 변환
                converted_names = batch_convert_drug_names(drug_codes, api_key_input)

                # 변환 결과를 세션에 저장
                st.session_state["converted_drug_names"] = converted_names

                # 클래스 딕셔너리 업데이트
                for idx, code in class_names_dict.items():
                    if code in converted_names:
                        class_names_dict[idx] = converted_names[code]

                st.sidebar.success(f"{len(converted_names)}개 약물명 변환 완료!")

                # 결과 미리보기
                with st.sidebar.expander("변환 결과 보기"):
                    for code, name in list(converted_names.items())[:5]:
                        st.text(f"{code} → {name}")
                    if len(converted_names) > 5:
                        st.text(f"... 외 {len(converted_names) - 5}개")
        else:
            st.sidebar.warning("API 키를 입력해주세요")
            st.sidebar.markdown(
                """
            **API 키 발급 방법:**
            1. [공공데이터포털](https://www.data.go.kr) 접속
            2. 회원가입 및 로그인
            3. '의약품 제품 정보' 검색
            4. 활용신청 후 API 키 발급
            """
            )

# Ollama 설정
st.sidebar.subheader("Ollama 설정")
ollama_url = st.sidebar.text_input(
    "Ollama API URL",
    value="http://localhost:11434/api/generate",
    help="Ollama 서버의 API 엔드포인트",
)

ollama_model = st.sidebar.selectbox(
    "Ollama 모델",
    ["llama2", "mistral", "llama3", "gemma", "phi"],
    help="사용할 Ollama 모델을 선택하세요",
)


# 모델 로드
@st.cache_resource
def load_model(model_path):
    try:
        model = YOLO(model_path)
        return model
    except Exception as e:
        st.error(f"모델 로드 실패: {e}")
        return None


# Ollama API 호출 함수
def call_ollama(prompt, model_name, api_url):
    """Ollama API를 호출하여 약물 상호작용 분석"""
    try:
        payload = {"model": model_name, "prompt": prompt, "stream": False}

        response = requests.post(api_url, json=payload, timeout=60)
        response.raise_for_status()

        result = response.json()
        return result.get("response", "응답을 받을 수 없습니다.")

    except requests.exceptions.ConnectionError:
        return (
            "오류: Ollama 서버에 연결할 수 없습니다. Ollama가 실행 중인지 확인하세요."
        )
    except requests.exceptions.Timeout:
        return "오류: 요청 시간이 초과되었습니다."
    except Exception as e:
        return f"오류: {str(e)}"


# 약물 상호작용 분석 프롬프트 생성
def create_drug_interaction_prompt(detected_drugs):
    """검출된 약물에 대한 상호작용 분석 프롬프트 생성"""

    drug_list = ", ".join(detected_drugs)

    prompt = f"""당신은 전문 약사입니다. 다음 약물들의 상호작용을 분석해주세요:

검출된 약물: {drug_list}
명확하고 이해하기 쉽게 한국어로 답변해주세요.
다음 형식으로 분석해주세요:

1. 약물 조합 평가
   - 이 약물들을 함께 복용해도 안전한가요?
   - 전체적인 위험도: [안전함/주의 필요/위험함]

2. 구체적인 상호작용
   - 긍정적 상호작용: 함께 복용하면 좋은 조합이 있나요?
   - 부정적 상호작용: 주의해야 할 조합이 있나요?

3. 복용 권장사항
   - 복용 시간대
   - 복용 순서
   - 기타 주의사항

4. 추가 조언
   - 의사나 약사와 상담이 필요한가요?
   - 특별히 주의해야 할 부작용

명확하고 이해하기 쉽게 한국어로 답변해주세요."""

    return prompt


# 한국의약품안전나라 API 연동 함수
@st.cache_data(ttl=3600)  # 1시간 캐싱
def get_drug_name_from_api(drug_code, api_key):
    """
    약물 식별번호를 실제 약물명으로 변환

    Args:
        drug_code: 약물 식별번호 (예: '1899')
        api_key: 의약품안전나라 API 키

    Returns:
        약물명 또는 None
    """
    try:
        # 의약품 제품정보 조회 API
        url = "http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList"

        params = {
            "serviceKey": api_key,
            "itemSeq": drug_code,  # 약물 식별번호
            "type": "json",
            "pageNo": "1",
            "numOfRows": "1",
        }

        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()

        data = response.json()

        # 응답 구조 파싱
        if "body" in data and "items" in data["body"]:
            items = data["body"]["items"]
            if items and len(items) > 0:
                item = items[0]
                # 제품명 반환
                return item.get("itemName", None)

        return None

    except Exception as e:
        st.warning(f"API 호출 실패 (약물코드: {drug_code}): {str(e)}")
        return None


def batch_convert_drug_names(drug_codes, api_key):
    """
    여러 약물 식별번호를 한번에 변환

    Args:
        drug_codes: 약물 식별번호 리스트
        api_key: API 키

    Returns:
        dict: {식별번호: 약물명} 매핑
    """
    result = {}

    with st.spinner(f"약물명 변환 중... (총 {len(drug_codes)}개)"):
        progress_bar = st.progress(0)

        for i, code in enumerate(drug_codes):
            drug_name = get_drug_name_from_api(str(code), api_key)
            if drug_name:
                result[str(code)] = drug_name
            else:
                result[str(code)] = f"약물_{code}"  # API 실패 시 기본값

            # 진행률 업데이트
            progress_bar.progress((i + 1) / len(drug_codes))

        progress_bar.empty()

    return result


# 모델 선택 방식
model_option = st.sidebar.radio(
    "모델 선택 방식",
    ["기본 모델 사용", "커스텀 모델 업로드"],
    help="기본 제공 모델을 사용하거나 직접 업로드할 수 있습니다",
)

model = None
model_path = None

if model_option == "기본 모델 사용":
    # model 폴더에서 .pt 파일 찾기
    model_dir = "/content/drive/MyDrive/Project-Team-1/data/yolo_results/yolov8n_train/weights"

    if os.path.exists(model_dir) and os.path.isdir(model_dir):
        # model 폴더의 모든 .pt 파일 찾기
        pt_files = [f for f in os.listdir(model_dir) if f.endswith(".pt")]

        if pt_files:
            # 발견된 모델 파일 선택
            selected_model = st.sidebar.selectbox(
                "모델 선택", pt_files, help="model 폴더에서 사용할 모델을 선택하세요"
            )

            model_path = os.path.join(model_dir, selected_model)

            # 모델 로드
            model = load_model(model_path)
            if model:
                st.sidebar.success(f"모델 로드 완료: {selected_model}")
            else:
                st.sidebar.error("모델 로드 실패")
        else:
            st.sidebar.error(f"'{model_dir}' 폴더에 .pt 파일이 없습니다")
    else:
        st.sidebar.error(f"'{model_dir}' 폴더를 찾을 수 없습니다")

else:  # 커스텀 모델 업로드
    uploaded_model = st.sidebar.file_uploader(
        "YOLO 모델 업로드 (.pt)",
        type=["pt"],
        help="학습된 YOLO 모델 파일을 업로드하세요",
    )

    if uploaded_model is not None:
        # 임시 파일로 저장
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp_file:
            tmp_file.write(uploaded_model.read())
            model_path = tmp_file.name

        # 모델 로드
        model = load_model(model_path)
        if model:
            st.sidebar.success(f"업로드된 모델 로드 완료: {uploaded_model.name}")
        else:
            st.sidebar.error("모델 로드 실패")
    else:
        st.sidebar.info("모델 파일을 업로드해주세요")

# 메인 영역
col1, col2 = st.columns(2)

with col1:
    st.subheader("이미지 업로드")
    uploaded_file = st.file_uploader(
        "이미지를 선택하세요",
        type=["jpg", "jpeg", "png"],
        help="JPG, JPEG, PNG 형식의 이미지를 업로드하세요",
    )

# 이미지가 업로드되고 모델이 로드되었을 때
if uploaded_file is not None and model is not None:
    # 이미지 읽기
    image = Image.open(uploaded_file)
    image_np = np.array(image)

    # 원본 이미지 표시
    with col1:
        st.image(image, caption="원본 이미지", use_container_width=True)

    # 검출 실행
    with st.spinner("약 검출 중..."):
        results = model(image_np, conf=confidence_threshold)

        # 결과 이미지 생성
        result_image = results[0].plot()
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

    # 결과 이미지 표시
    with col2:
        st.subheader("검출 결과")
        st.image(result_image, caption="검출된 약", use_container_width=True)

    # 검출된 객체 정보
    st.subheader("검출 상세 정보")

    detections = results[0].boxes
    if len(detections) > 0:
        st.success(f"총 {len(detections)}개의 약이 검출되었습니다.")

        # 검출 정보 테이블
        detection_data = []
        detected_drug_names = []

        for i, box in enumerate(detections):
            class_id = int(box.cls[0])
            confidence = float(box.conf[0])

            # 약물명 결정 우선순위
            if class_name_option != "모델 기본값" and class_id in class_names_dict:
                # 업로드된 파일이나 직접 입력한 이름 사용
                class_name = class_names_dict[class_id]

                # API로 변환된 이름이 있으면 사용
                if "converted_drug_names" in st.session_state:
                    converted = st.session_state["converted_drug_names"]
                    if class_name in converted:
                        class_name = converted[class_name]

            elif hasattr(model, "names") and class_id in model.names:
                # 모델에 정의된 이름 사용
                class_name = model.names[class_id]
            else:
                # 기본값
                class_name = f"약물 {class_id}"

            detection_data.append(
                {
                    "번호": i + 1,
                    "약물명": class_name,
                    "클래스 ID": class_id,
                    "신뢰도": f"{confidence:.2%}",
                }
            )

            detected_drug_names.append(class_name)

        st.table(detection_data)

        # 약물 상호작용 분석 섹션
        st.subheader("약물 상호작용 분석")

        if len(detected_drug_names) >= 2:
            if st.button("약물 상호작용 분석 시작", type="primary"):
                with st.spinner("Ollama를 통해 약물 상호작용을 분석하는 중..."):
                    # 프롬프트 생성
                    prompt = create_drug_interaction_prompt(detected_drug_names)

                    # Ollama API 호출
                    analysis_result = call_ollama(prompt, ollama_model, ollama_url)

                    # 결과 표시
                    st.markdown("### 분석 결과")
                    st.markdown(analysis_result)

                    # 프롬프트 보기 (디버깅용)
                    with st.expander("사용된 프롬프트 보기"):
                        st.code(prompt, language="text")

        elif len(detected_drug_names) == 1:
            st.info("약물 상호작용 분석을 위해서는 2개 이상의 약이 필요합니다.")

        # 결과 다운로드
        st.subheader("결과 다운로드")
        result_pil = Image.fromarray(result_image)

        # 이미지를 메모리 내 버퍼에 저장
        buf = io.BytesIO()
        result_pil.save(buf, format="JPEG")
        buf.seek(0)

        st.download_button(
            label="결과 이미지 다운로드",
            data=buf,
            file_name="detected_pills.jpg",
            mime="image/jpeg",
        )
    else:
        st.warning("검출된 약이 없습니다. 신뢰도 임계값을 낮춰보세요.")

elif uploaded_file is None:
    st.info("이미지를 업로드하여 약 검출을 시작하세요.")
elif model is None:
    st.error("모델을 먼저 로드해주세요.")

# 사용 방법
with st.expander("사용 방법"):
    st.markdown(
        """
    ### YOLO 약 검출

    **모델 준비**
    1. 방법 1 - 기본 모델 사용:
       - `model` 폴더에 .pt 파일 넣기
       - 사이드바에서 "기본 모델 사용" 선택
       - 드롭다운에서 모델 선택

    2. 방법 2 - 커스텀 모델 업로드:
       - 사이드바에서 "커스텀 모델 업로드" 선택
       - .pt 파일 직접 업로드

    **약물명 설정**

    약물명이 숫자나 식별번호로 나오는 경우 다음 방법 중 선택:

    1. **YAML/JSON 파일 업로드 (권장)**
       - YOLO 학습 시 사용한 YAML 파일 업로드
       - 또는 클래스 매핑 JSON 파일 업로드
       - 자동으로 클래스명이 로드됩니다

    2. **직접 입력**
       - 텍스트 영역에 약물명 입력 (쉼표로 구분)
       - 예: 타이레놀, 아스피린, 게보린

    3. **모델 기본값**
       - 모델에 저장된 이름 사용

    **검출 실행**
    1. 검출 신뢰도 임계값을 조정하세요 (기본값: 0.25)
    2. 약이 포함된 이미지를 업로드하세요
    3. 검출 결과를 확인하세요

    ### Ollama 약물 상호작용 분석
    1. Ollama를 로컬에 설치하고 실행하세요
       ```bash
       # Ollama 설치 (Linux/Mac)
       curl -fsSL https://ollama.com/install.sh | sh

       # 모델 다운로드
       ollama pull llama2

       # Ollama 서버 실행
       ollama serve
       ```
    2. 사이드바에서 Ollama 모델을 선택하세요
    3. 2개 이상의 약이 검출되면 "약물 상호작용 분석 시작" 버튼을 클릭하세요
    4. AI가 약물 간 상호작용을 분석한 결과를 확인하세요

    ### 주의사항
    - 약물 식별번호를 실제 약물명으로 변환하려면 별도의 매핑 파일이 필요합니다
    - 이 분석 결과는 참고용이며, 실제 복용 전 반드시 의사나 약사와 상담하세요
    - Ollama 서버가 실행 중이어야 상호작용 분석이 가능합니다
    """
    )

# 프롬프트 엔지니어링 팁
with st.expander("프롬프트 엔지니어링 가이드"):
    st.markdown(
        """
    ### 현재 프롬프트 구조

    이 앱은 다음과 같은 프롬프트 엔지니어링 기법을 사용합니다:

    1. **역할 부여 (Role Assignment)**
       - "당신은 전문 약사입니다"로 시작하여 AI에게 전문가 역할 부여

    2. **구조화된 출력 (Structured Output)**
       - 명확한 번호와 카테고리로 응답 형식 지정
       - 평가, 상호작용, 권장사항, 조언으로 구분

    3. **구체적인 지시 (Specific Instructions)**
       - "명확하고 이해하기 쉽게"와 같은 구체적 요구사항
       - 위험도 수준을 명시적으로 요청

    4. **컨텍스트 제공 (Context Provision)**
       - 검출된 약물 목록을 명확히 제공
       - 분석해야 할 항목을 구체적으로 나열

    ### 프롬프트 커스터마이징

    코드에서 `create_drug_interaction_prompt` 함수를 수정하여:
    - 더 상세한 분석 요청
    - 특정 환자 상황 고려
    - 다른 형식의 출력 요청
    등을 추가할 수 있습니다.
    """
    )


Writing pill_detection_ollama_appl.py


## 섹션 3 StreamLit 구동

In [3]:

from pyngrok import ngrok
import subprocess
import time
import sys


PORT = 8508

# 1. ngrok 기존 터널 정리 (안정성 향상)
try:
    ngrok.kill()
    print("기존 ngrok 프로세스를 종료했습니다.")
except:
    pass # ngrok이 실행되고 있지 않아도 무시

# 2. Streamlit 실행 (백그라운드)
# Streamlit 실행 시 포트  헤드리스 모드 설정
# stdout/stderr를 파이프로 연결하여 로그를 나중에 확인할 수 있도록 합니다.
try:
    streamlit_command = [
        "streamlit",
        "run",
        "pill_detection_ollama_appl.py",
        "--server.port", str(PORT),
        "--server.headless", "true"
    ]
    p = subprocess.Popen(
        streamlit_command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    print(f"\nStreamlit 앱({streamlit_command[2]})을 포트 {PORT}에서 시작 중입니다...")

    # 🚀 중요: Streamlit이 완전히 시작될 때까지 충분히 대기합니다.
    # YOLO 모델 로딩 등으로 인해 10초 대기가 필요할 수 있습니다.
    time.sleep(10)
    print("대기 시간 종료. ngrok 연결을 시도합니다...")

except FileNotFoundError:
    print("\n❌ 오류: 'streamlit' 명령어를 찾을 수 없습니다. Streamlit이 설치되었는지 확인하세요.")
    sys.exit(1)
except Exception as e:
    print(f"\n❌ Streamlit 실행 중 예기치 않은 오류 발생: {e}")
    sys.exit(1)


# 3. ngrok 인증 토큰 설정
NGROK_AUTH_TOKEN = "36uCNORs8jUrEjJGOEf05Cyrc5V_7Ae1HknTvrbmjLc4i3Ni6"
try:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except Exception as e:
    print(f"\n❌ ngrok 인증 토큰 설정 오류: {e}")
    p.terminate()
    sys.exit(1)


# 4. ngrok 터널 열기 및 결과 출력
public_url = None
try:

    public_url = ngrok.connect(PORT)

    print("\n✅ 터널 연결 성공!")
    print(f"Streamlit App이 실행 중입니다. 다음 URL로 접속하세요:")
    print(f"🔗 {public_url}")

except Exception as e:
    print(f"\n❌ ngrok 연결 중 치명적인 오류 발생: {e}")
    print(f"ngrok 연결 실패! Streamlit이 {PORT} 포트에서 실행되지 않았을 수 있습니다.")

finally:
    # 5. Streamlit 프로세스 상태 및 로그 확인
    print("\n--- Streamlit 프로세스 상태 및 로그 확인 ---")

    # communicate()를 호출하기 전에 프로세스가 종료되었는지 확인 (비종료 시 오류가 발생할 수 있습니다.)
    if p.poll() is None and public_url is not None:
        print("💡 Streamlit 프로세스가 현재 정상적으로 실행 중입니다. (p.poll() == None)")
        print("앱이 정상적으로 로드되지 않는다면, 앱 내부의 초기화 지연 때문일 수 있습니다.")
        print("터널이 열려 있으므로, 이 셀을 계속 실행 상태로 두세요.")

    elif p.poll() is not None:
        print(f"❌ Streamlit 프로세스가 종료되었습니다. 종료 코드: {p.returncode}")
        # 프로세스가 종료되었다면 communicate()를 호출하여 남은 로그를 확인
        try:
            # timeout을 사용하여 강제로 로그를 읽어옵니다.
            stdout_output, stderr_output = p.communicate(timeout=1)

            print("\n--- Streamlit 표준 출력 (STDOUT) ---")
            print(stdout_output.decode('utf-8'))

            print("\n--- Streamlit 오류 출력 (STDERR) ---")
            # STDERR에 Python Traceback (실제 오류)이 포함될 가능성이 높습니다.
            print(stderr_output.decode('utf-8'))

        except subprocess.TimeoutExpired:
             # 드물지만, 통신 시도 중 타임아웃이 발생할 경우
            print("로그 확인 중 타임아웃이 발생했습니다.")

    else:
        # ngrok 연결에 실패했고, 프로세스도 종료되지 않았을 경우
        print("ngrok 연결은 실패했지만 Streamlit 프로세스는 여전히 실행 중일 수 있습니다.")
        print("직접 프로세스 p를 확인하거나, 'ngrok.kill()'을 다시 실행하여 모든 연결을 정리하세요.")

# 만약 이 셀을 실행한 후에도 브라우저에서 로드되지 않는다면,
# 위의 '--- Streamlit 오류 출력 (STDERR) ---' 부분에 나타난 오류를 확인해 주세요.

#ngrok.set_auth_token("36uCNORs8jUrEjJGOEf05Cyrc5V_7Ae1HknTvrbmjLc4i3Ni6")


기존 ngrok 프로세스를 종료했습니다.

Streamlit 앱(pill_detection_ollama_appl.py)을 포트 8508에서 시작 중입니다...
대기 시간 종료. ngrok 연결을 시도합니다...

✅ 터널 연결 성공!
Streamlit App이 실행 중입니다. 다음 URL로 접속하세요:
🔗 NgrokTunnel: "https://contextually-unmediaeval-kaleb.ngrok-free.dev" -> "http://localhost:8508"

--- Streamlit 프로세스 상태 및 로그 확인 ---
💡 Streamlit 프로세스가 현재 정상적으로 실행 중입니다. (p.poll() == None)
앱이 정상적으로 로드되지 않는다면, 앱 내부의 초기화 지연 때문일 수 있습니다.
터널이 열려 있으므로, 이 셀을 계속 실행 상태로 두세요.
